<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_universe_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_universe_ai.py

End-to-end pipeline for UniverseAI:
1. Synthetic dataset of 6 “universe” features → 3 physics targets
2. Float32 normalization
3. MLP with LayerNorm, Dropout & ReLU
4. Physics-informed constraint enforcing toy balance laws
5. MC-Dropout for uncertainty quantification
6. Training loop with AdamW, ReduceLROnPlateau, gradient clipping, NaN checks, early stopping
7. Checkpointing and safe loading
8. Visualizations: loss curves, scatter plots, uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

# ------------------------------------------------------------------------------
# 1. Synthetic Universe Dataset
# ------------------------------------------------------------------------------
class UniverseDataset(Dataset):
    def __init__(self, n_samples=5000, seed=42):
        np.random.seed(seed)
        # Features:
        # TF: time flow rate ∈ [0.5, 2.0]
        # SE: space expansion factor ∈ [0.1, 3.0]
        # FF: fundamental forces balance metric ∈ [0.1, 1.0]
        # GC: gravitational constant factor ∈ [0.1, 2.0]
        # CC: cosmological constant ∈ [1e-12, 1e-10]
        # IE: initial entropy ∈ [1.0, 100.0]
        TF = np.random.uniform(0.5, 2.0,   (n_samples,1))
        SE = np.random.uniform(0.1, 3.0,   (n_samples,1))
        FF = np.random.uniform(0.1, 1.0,   (n_samples,1))
        GC = np.random.uniform(0.1, 2.0,   (n_samples,1))
        CC = np.random.uniform(1e-12,1e-10,(n_samples,1))
        IE = np.random.uniform(1.0, 100.0, (n_samples,1))

        X_raw = np.hstack([TF, SE, FF, GC, CC, IE]).astype(np.float64)

        # Toy physics targets:
        # UC: universe coherence ∝ TF * FF / (CC + eps)
        # ES: entropy stability ∝ IE / (SE + eps)
        # TB: time-energy balance ∝ TF * IE / (GC + eps)
        eps = 1e-12
        UC = TF * FF / (CC + eps)
        ES = IE / (SE + eps)
        TB = TF * IE / (GC + eps)

        Y_raw = np.hstack([UC, ES, TB]).astype(np.float64)
        # add small noise
        Y_raw += 0.02 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape)

        # compute normalization stats
        self.X_mean = X_raw.mean(axis=0)
        self.X_std  = X_raw.std(axis=0) + 1e-8
        self.Y_mean = Y_raw.mean(axis=0)
        self.Y_std  = Y_raw.std(axis=0) + 1e-8

        # normalize and cast to float32
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.Y[idx])

# ------------------------------------------------------------------------------
# 2. UniverseAI Model Definition
# ------------------------------------------------------------------------------
class UniverseAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dims=(64,64), output_dim=3, p_drop=0.1):
        super().__init__()
        layers, dim = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics-Informed Constraint & Loss
# ------------------------------------------------------------------------------
def physics_constraint(pred, X, stats):
    X_den = X * stats['X_std'] + stats['X_mean']
    TF, SE, FF, GC, CC, IE = X_den.t()
    eps = 1e-12

    UC_t = TF * FF / (CC + eps)
    ES_t = IE / (SE + eps)
    TB_t = TF * IE / (GC + eps)

    Yt = torch.stack([UC_t, ES_t, TB_t], dim=1)
    Yt_norm = (Yt - stats['Y_mean']) / stats['Y_std']
    return nn.MSELoss()(pred, Yt_norm)

def total_loss(pred, true, X, stats, weight=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_constraint(pred, X, stats)
    return mse + weight * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC-Dropout for Uncertainty Quantification
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    arr = torch.stack(preds, dim=0)
    return arr.mean(0), arr.std(0)

# ------------------------------------------------------------------------------
# 5. Training Loop with Safety & Checkpointing
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-4, wd=1e-5, epochs=100, patience=10, phys_weight=1.0):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    best_val, wait = float('inf'), 0
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(1, epochs+1):
        model.train()
        total_train_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, _, _ = total_loss(pred, yb, xb, stats, phys_weight)
            if torch.isnan(loss):
                print(f"NaN encountered at epoch {epoch}, stopping.")
                return history
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_train_loss += loss.item() * xb.size(0)

        train_loss = total_train_loss / len(train_loader.dataset)

        # validation
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, _, _ = total_loss(pred, yb, xb, stats, phys_weight)
                total_val_loss += loss.item() * xb.size(0)

        val_loss = total_val_loss / len(val_loader.dataset)
        scheduler.step(val_loss)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        print(f"Epoch {epoch:03d} → Train: {train_loss:.4e}, Val: {val_loss:.4e}")

        # checkpoint & early stop
        if val_loss < best_val - 1e-8:
            best_val, wait = val_loss, 0
            torch.save(model.state_dict(), "best_universe_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    if os.path.exists("best_universe_ai.pth"):
        model.load_state_dict(torch.load("best_universe_ai.pth", map_location=device))

    return history

# ------------------------------------------------------------------------------
# 6. Visualization Utilities
# ------------------------------------------------------------------------------
def plot_history(history):
    plt.figure(figsize=(6,4))
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'],   label='Val Loss')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training History")
    plt.show()

def plot_scatter(y_true, y_pred, name):
    plt.figure(figsize=(4,4))
    plt.scatter(y_true, y_pred, alpha=0.6, s=8)
    m, M = y_true.min(), y_true.max()
    plt.plot([m,M],[m,M],'r--')
    plt.title(name)
    plt.xlabel("True")
    plt.ylabel("Predicted")
    plt.show()

def plot_uncertainty_heatmap(model, stats, device):
    grid = 100
    SE_space = np.linspace(0.1, 3.0, grid, dtype=np.float32)
    IE_space = np.linspace(1.0, 100.0, grid, dtype=np.float32)
    G1, G2 = np.meshgrid(SE_space, IE_space)
    points = G1.size

    Xg = torch.zeros((points, 6), device=device)
    for i in (0,2,3,4):  # TF, FF, GC, CC = mean
        Xg[:,i] = stats['X_mean'][i]
    Xg[:,1] = torch.from_numpy(G1.ravel()).to(device)  # SE
    Xg[:,5] = torch.from_numpy(G2.ravel()).to(device)  # IE

    Xn = (Xg - stats['X_mean']) / stats['X_std']
    _, std = mc_dropout_predict(model, Xn, T=100)
    U = std[:,0].cpu().reshape(G1.shape)

    plt.figure(figsize=(5,4))
    plt.pcolormesh(G1, G2, U, cmap='magma', shading='auto')
    plt.colorbar(label='Uncertainty: Universe Coherence')
    plt.xlabel('Space Expansion Factor')
    plt.ylabel('Initial Entropy')
    plt.title('Uncertainty Heatmap')
    plt.tight_layout()
    plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset = UniverseDataset(n_samples=5000, seed=42)
    stats = {
        'X_mean': torch.tensor(dataset.X_mean, dtype=torch.float32, device=device),
        'X_std' : torch.tensor(dataset.X_std,  dtype=torch.float32, device=device),
        'Y_mean': torch.tensor(dataset.Y_mean, dtype=torch.float32, device=device),
        'Y_std' : torch.tensor(dataset.Y_std,  dtype=torch.float32, device=device),
    }

    n_val = int(0.2 * len(dataset))
    train_ds, val_ds = random_split(dataset, [len(dataset)-n_val, n_val])
    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=256, shuffle=False)

    model = UniverseAI(input_dim=6, hidden_dims=(32,32), output_dim=3, p_drop=0.1).to(device)
    history = train(model, train_loader, val_loader, stats, device,
                    lr=1e-4, wd=1e-5, epochs=100, patience=10, phys_weight=1.0)

    plot_history(history)

    X_all = torch.from_numpy(dataset.X).to(device)
    with torch.no_grad():
        Yp_norm = model(X_all).cpu().numpy()
    Yp = Yp_norm * dataset.Y_std + dataset.Y_mean
    Yt = dataset.Y * dataset.Y_std + dataset.Y_mean

    names = ["Universe Coherence", "Entropy Stability", "Time-Energy Balance"]
    for i, nm in enumerate(names):
        plot_scatter(Yt[:,i], Yp[:,i], nm)

    plot_uncertainty_heatmap(model, stats, device)